In [2]:
from langchain_community.tools import WikipediaQueryRun,ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [3]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [6]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [7]:
tools=[wiki,arxiv]

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

f:\Langchain\LangchainProjects\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
loader=WebBaseLoader(
    "https://docs.smith.langchain.com/"
)
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vecotdb=FAISS.from_documents(documents,embeddings)
retriever=vecotdb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002601BE5E530>, search_kwargs={})

In [20]:
retriever.invoke("how to upload a dataset")

[Document(id='dc5dd713-8827-45f4-b513-e3f5b43f6cfa', metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Run the evaluationexperiment_results = client.evaluate(    dummy_app, # Your AI system goes here    data=dataset, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={"version": "1.0.0", "revision_id": "beta"}, # Metadata about the experiment    max_concurrency=4,  # Add concurrency.)# Analyze the results via the UI or programmatically# If you have \'pandas\' installed you can view the results as a# pandas DataFrame by uncommenting below:# experiment_results.to_pandas()import { Client } from "langsmith";import { EvaluationResult, evaluate } from "langsmith/evaluation";c

In [19]:
#convert retriever as tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [21]:
retriever_tool.invoke("how to upload a dataset")

'Run the evaluationexperiment_results = client.evaluate(    dummy_app, # Your AI system goes here    data=dataset, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={"version": "1.0.0", "revision_id": "beta"}, # Metadata about the experiment    max_concurrency=4,  # Add concurrency.)# Analyze the results via the UI or programmatically# If you have \'pandas\' installed you can view the results as a# pandas DataFrame by uncommenting below:# experiment_results.to_pandas()import { Client } from "langsmith";import { EvaluationResult, evaluate } from "langsmith/evaluation";const client = new Client();// Define dataset: these are your test casesconst datasetName = "Sample Dataset";const dataset = await client.createDataset(datasetName, {  description: "A sample dataset in LangSmith.",});await client.createExamples({  inputs: [    { postfix: "to LangSmith" },

In [22]:
tools=[wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'f:\\Langchain\\LangchainProjects\\.venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002607ACD0F70>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain

In [23]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
llm=ChatGroq(model='Llama3-8b-8192')

In [24]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [28]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [34]:
from langchain.agents import create_tool_calling_agent

agent=create_tool_calling_agent(llm,tools,prompt)

In [36]:
from langchain.agents import AgentExecutor

agents_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [37]:
agents_executor.invoke({"input":"what is Machine Leanring"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks wit
Invoking: `arxiv` with `{'query': 'Machine learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton University and tutorials given in MLSS, Buenos Aires, as
Invoking: `langsmith_search` with `{'query': 'Machine learning'}`


Trace LLM Applications: Gain visibility into LLM calls and other parts of your application's logic.
Evaluate Performance: Compare results across models, prompts, and architectures to identify what works best.
Improve Prompts: Quickly refine prompts to achieve more accura

{'input': 'what is Machine Leanring',
 'output': 'Based on the provided information, it seems that the topic of Machine Learning has branched off into a discussion about LangSmith and its related tools. Since the provided information is not directly related to the original topic of Machine Learning, I will provide a direct response.\n\nMachine Learning is a field of study in Artificial Intelligence that involves the use of algorithms and statistical models to enable machines to learn from data and make predictions or decisions without being explicitly programmed.'}